In [1]:
import numpy as np
import pandas as pd
import os

from common import OUTPUTPATH
from models import rule_fit_pcc
import data

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)


/var/folders/c_/6873d_hj27z4bhy1l9x1tggnj8s0wd/T/ipykernel_82533/2424450009.py:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


This notebook refit the models in the all dataset

Fit all data takes ~3 mim under 2.6 GHz 6-Core Intel Core i7

In [2]:
rule_fit_pcc.fit(data.x1, data.y)

In [3]:
fitted_model = rule_fit_pcc.fitted_
mophology = ['Sphere', 'Worm', 'Vesicle', 'Other']
for i in range(len(fitted_model)):
    print(mophology[i], 'Cs:', fitted_model[i].model.Cs)

Sphere Cs: [1]
Worm Cs: [1]
Vesicle Cs: [1]
Other Cs: [32]


Get the individual morphology rules

In [4]:
all_columns = data.predictors + data.targets
rule_sphere = rule_fit_pcc.fitted_[0].get_rules(all_columns)[['rule','type','coef','importance']]
rule_sphere.to_csv(os.path.join(OUTPUTPATH, 'Sphere_rules.csv'), index=False)
rule_sphere

,rule,type,coef,importance
9318,dp_core> 132.5 & mon_core_mv> 1.0413 & clogp_core<= 1.37 & mon_core_mv<= 1.1928 & dp_corona<= 64.5 & core_mw_total> 21611.625 & ratio_vol<= 0.2233 & mon_corona_apol<= 27.5791,rule,-2.320588,0.750651
8748,corona_mw_total<= 11925.2148 & mon_corona_apol<= 23.6109 & core_mw_total> 15703.46 & clogp_core<= 1.37 & dp_corona<= 63.5 & dp_core> 96.5 & mon_corona_mv<= 1.344 & core_mv_total<= 510.1229 & corona_mv_total<= 67.7817,rule,-1.996514,0.632467
2235,conc<= 15.5 & ratio_mass<= 0.4196 & mon_core_mw<= 145.3096 & ratio_vol<= 0.6272 & conc<= 16.5 & mon_core_mw<= 144.5526 & corona_mw_total> 10396.3247,rule,1.671920,0.533947
3564,ratio_vol<= 0.2732 & corona_mw_total<= 12760.5298 & clogp_core<= 1.37 & mon_core_psa> 46.1575 & mon_core_psa> 46.35 & core_mv_total> 141.3628 & ratio_vol<= 0.3104 & conc> 16.1 & ratio_mass> 0.104 & clogp_corona<= 0.125 & mon_corona_mw<= 287.115,rule,-1.699665,0.527928
6441,ratio_vol<= 0.3004 & mon_core_psa> 30.915 & clogp_core<= 1.37 & mon_corona_apol<= 58.2307 & mon_corona_mv> 1.3573 & ratio_mass> 0.1869 & dp_corona> 28.0 & dp_corona<= 64.5 & corona_mv_total<= 78.3154,rule,-2.869156,0.521525
10596,mon_corona_mw> 163.9357 & conc> 15.5 & core_mw_total> 30761.0801 & clogp_corona<= -2.73 & mon_corona_psa<= 215.065 & mon_corona_mv<= 2.3358 & conc> 16.1 & ratio_vol<= 0.2996 & clogp_core> 0.895 & core_mw_total> 26510.7197,rule,-2.057515,0.434709
12959,core_mw_total> 27374.7246 & corona_mw_total<= 11925.2148 & corona_mw_total<= 12760.5298 & temp> 47.0 & clogp_core<= 1.37 & conc> 6.25 & ratio_vol<= 0.3038 & ratio_mass<= 0.2783 & mon_corona_apol<= 58.2307 & mon_core_mw<= 145.3096 & clogp_corona> -5.23 & salt<= 0.05 & charged<= 0.5,rule,-1.265794,0.396393
1758,conc<= 27.5 & mon_corona_psa> 58.045 & corona_mw_total<= 12760.5298 & corona_mw_total<= 10396.3247 & mon_core_mw<= 145.3096 & mon_corona_mv<= 1.3573 & charged<= 0.5 & ratio_mass<= 0.3269,rule,-1.149066,0.380018
4901,conc> 14.25 & ratio_vol<= 0.3025 & dp_corona> 28.5 & core_mw_total> 15703.46 & corona_mw_total<= 10497.1396 & mon_core_psa> 40.26 & clogp_corona<= 0.605 & core_mw_total<= 39320.0 & core_mv_total> 152.3496 & dp_corona<= 62.5,rule,-1.164806,0.370700
7291,core_mv_total<= 296.8425 & clogp_core<= 1.18 & conc<= 27.5 & dp_corona<= 87.5 & dp_core<= 185.0 & mon_core_mw<= 145.3096 & ratio_vol<= 0.3115 & mon_corona_mv<= 1.3573 & ratio_vol> 0.2368 & mon_core_mw<= 144.4899,rule,-1.327200,0.369538


In [5]:
all_columns = data.predictors + data.targets
rule_sphere = rule_fit_pcc.fitted_[0].get_rules(all_columns)[['rule','type','coef','importance']]
rule_sphere.to_csv(os.path.join(OUTPUTPATH, 'Sphere_rules.csv'), index=False)
rule_sphere

,rule,type,coef,importance
9318,dp_core> 132.5 & mon_core_mv> 1.0413 & clogp_core<= 1.37 & mon_core_mv<= 1.1928 & dp_corona<= 64.5 & core_mw_total> 21611.625 & ratio_vol<= 0.2233 & mon_corona_apol<= 27.5791,rule,-2.320588,0.750651
8748,corona_mw_total<= 11925.2148 & mon_corona_apol<= 23.6109 & core_mw_total> 15703.46 & clogp_core<= 1.37 & dp_corona<= 63.5 & dp_core> 96.5 & mon_corona_mv<= 1.344 & core_mv_total<= 510.1229 & corona_mv_total<= 67.7817,rule,-1.996514,0.632467
2235,conc<= 15.5 & ratio_mass<= 0.4196 & mon_core_mw<= 145.3096 & ratio_vol<= 0.6272 & conc<= 16.5 & mon_core_mw<= 144.5526 & corona_mw_total> 10396.3247,rule,1.671920,0.533947
3564,ratio_vol<= 0.2732 & corona_mw_total<= 12760.5298 & clogp_core<= 1.37 & mon_core_psa> 46.1575 & mon_core_psa> 46.35 & core_mv_total> 141.3628 & ratio_vol<= 0.3104 & conc> 16.1 & ratio_mass> 0.104 & clogp_corona<= 0.125 & mon_corona_mw<= 287.115,rule,-1.699665,0.527928
6441,ratio_vol<= 0.3004 & mon_core_psa> 30.915 & clogp_core<= 1.37 & mon_corona_apol<= 58.2307 & mon_corona_mv> 1.3573 & ratio_mass> 0.1869 & dp_corona> 28.0 & dp_corona<= 64.5 & corona_mv_total<= 78.3154,rule,-2.869156,0.521525
10596,mon_corona_mw> 163.9357 & conc> 15.5 & core_mw_total> 30761.0801 & clogp_corona<= -2.73 & mon_corona_psa<= 215.065 & mon_corona_mv<= 2.3358 & conc> 16.1 & ratio_vol<= 0.2996 & clogp_core> 0.895 & core_mw_total> 26510.7197,rule,-2.057515,0.434709
12959,core_mw_total> 27374.7246 & corona_mw_total<= 11925.2148 & corona_mw_total<= 12760.5298 & temp> 47.0 & clogp_core<= 1.37 & conc> 6.25 & ratio_vol<= 0.3038 & ratio_mass<= 0.2783 & mon_corona_apol<= 58.2307 & mon_core_mw<= 145.3096 & clogp_corona> -5.23 & salt<= 0.05 & charged<= 0.5,rule,-1.265794,0.396393
1758,conc<= 27.5 & mon_corona_psa> 58.045 & corona_mw_total<= 12760.5298 & corona_mw_total<= 10396.3247 & mon_core_mw<= 145.3096 & mon_corona_mv<= 1.3573 & charged<= 0.5 & ratio_mass<= 0.3269,rule,-1.149066,0.380018
4901,conc> 14.25 & ratio_vol<= 0.3025 & dp_corona> 28.5 & core_mw_total> 15703.46 & corona_mw_total<= 10497.1396 & mon_core_psa> 40.26 & clogp_corona<= 0.605 & core_mw_total<= 39320.0 & core_mv_total> 152.3496 & dp_corona<= 62.5,rule,-1.164806,0.370700
7291,core_mv_total<= 296.8425 & clogp_core<= 1.18 & conc<= 27.5 & dp_corona<= 87.5 & dp_core<= 185.0 & mon_core_mw<= 145.3096 & ratio_vol<= 0.3115 & mon_corona_mv<= 1.3573 & ratio_vol> 0.2368 & mon_core_mw<= 144.4899,rule,-1.327200,0.369538


In [6]:
rule_worm = rule_fit_pcc.fitted_[1].get_rules(all_columns)[['rule','type','coef','importance']]
rule_worm.to_csv(os.path.join(OUTPUTPATH, 'Worm_rules.csv'), index=False)
rule_worm

,rule,type,coef,importance
1589,dp_corona> 27.0 & ratio_vol> 0.2022 & temp> 47.0 & ph> 5.5 & ratio_vol> 0.2135 & corona_mv_total> 79.0114 & conc> 11.5 & dp_core> 157.5 & ratio_mass<= 0.4183 & dp_corona<= 88.0 & mon_core_mw<= 156.06,rule,2.954974,0.846445
5854,mon_core_psa> 37.18 & mon_corona_apol> 11.8462 & clogp_core<= 1.4 & clogp_core<= 1.15 & ratio_mass> 0.1534 & dp_core<= 146.5 & ratio_mass<= 0.4161 & corona_mv_total<= 52.4219 & core_mw_total<= 43151.2109 & corona_mw_total<= 7143.2949 & ratio_mass> 0.1356 & corona_mv_total<= 69.0254,rule,3.577500,0.789602
4306,clogp_core<= 1.18 & mon_core_psa<= 46.7805 & dp_core> 200.5 & conc<= 11.25 & dp_core> 167.5,rule,-1.810239,0.625761
7624,dp_core> 299.5 & ratio_vol<= 0.19 & dp_corona> 25.5 & ratio_mass<= 0.2363,rule,-1.130413,0.489483
6235,core_mv_total> 133.3125 & ratio_vol> 0.2088 & mon_core_apol> 21.0474 & clogp_core<= 1.37 & corona_mw_total<= 16666.9927 & ratio_mass> 0.2346 & corona_mv_total> 66.2538 & sphere<= 0.5,rule,1.488611,0.482079
9452,ratio_mass<= 0.3258 & mon_corona_psa> 56.685 & mon_core_psa> 37.18 & core_mw_total<= 27668.5752 & mon_core_mw> 136.06 & dp_core<= 299.5 & ratio_vol> 0.2471,rule,1.504988,0.448277
9785,dp_core> 177.5 & core_mw_total> 23506.29 & dp_corona> 59.0 & ratio_mass<= 0.2335 & clogp_corona> -0.1361 & core_mw_total<= 32778.1992 & core_mw_total> 17682.2754,rule,2.008354,0.362763
10376,corona_mw_total> 12760.5298 & ratio_vol> 0.1934,rule,-1.452091,0.355846
12404,clogp_corona> -4.035 & corona_mw_total<= 12760.5298 & mon_core_psa<= 47.1448 & corona_mw_total> 7450.645 & dp_corona<= 63.5 & ratio_mass> 0.248 & dp_core<= 299.0 & clogp_core<= 1.36 & mon_core_psa<= 46.7349 & ratio_mass> 0.2346 & ratio_vol<= 0.3403 & dp_core> 102.0 & mon_core_mv> 1.1037,rule,1.082204,0.338901
12438,ratio_vol> 0.1042 & dp_corona<= 105.0 & core_mw_total> 26242.335 & mon_corona_psa<= 93.29 & ratio_mass<= 0.2235 & conc> 11.25,rule,-0.941023,0.328523


In [7]:
rule_vesicle = rule_fit_pcc.fitted_[2].get_rules(all_columns)[['rule','type','coef','importance']]
rule_vesicle.to_csv(os.path.join(OUTPUTPATH, 'Vesicle_rules.csv'), index=False)
rule_vesicle

,rule,type,coef,importance
747,corona_mw_total<= 12760.5298 & core_mw_total> 31657.25 & conc> 13.25 & mon_core_mv> 1.1145 & clogp_core<= 1.16 & ratio_mass<= 0.245 & corona_mw_total> 8883.8848,rule,1.663219,0.502504
3272,corona_mw_total<= 7923.4648 & sphere<= 0.5 & dp_core> 54.0 & ph<= 3.85,rule,1.708759,0.461175
1953,conc> 13.75 & ratio_mass<= 0.2535 & clogp_core<= 1.35 & mon_corona_psa<= 79.32 & core_mw_total> 17671.9951 & temp> 65.0 & sphere<= 0.5,rule,1.552364,0.425836
7803,ratio_mass> 0.1523 & conc<= 27.5 & sphere> 0.5 & core_mv_total<= 314.6175,rule,-0.830986,0.388729
4728,mon_core_mw> 136.8175 & corona_mw_total> 4260.0101 & clogp_core<= 1.18 & temp> 53.0 & corona_mv_total<= 107.5755 & ratio_mass<= 0.2467 & mon_corona_psa<= 93.29 & ratio_mass<= 0.251 & clogp_corona<= 0.215 & salt<= 0.15 & ratio_mass> 0.0452 & sphere<= 0.5,rule,0.956231,0.337491
4497,worm<= 0.5 & temp> 47.0 & core_mv_total<= 1374.6 & dp_core> 60.0 & mon_core_apol> 21.894 & mon_core_apol<= 22.7646 & corona_mv_total> 25.3733 & sphere<= 0.5,rule,0.844196,0.305583
7066,dp_core<= 167.5 & ph> 4.0,rule,-0.674638,0.300048
8873,worm<= 0.5 & clogp_core<= 1.37 & mon_core_mw> 142.8175 & corona_mw_total<= 14063.4102 & mon_corona_psa> 47.97 & ratio_mass> 0.0551 & sphere<= 0.5,rule,0.898378,0.296100
8164,mon_core_psa<= 47.1448 & ratio_vol<= 0.2613 & conc> 7.5 & sphere> 0.5 & corona_mw_total<= 10526.5049,rule,-0.719957,0.293079
5966,conc<= 12.75 & mon_corona_psa<= 130.3 & ratio_vol> 0.149 & charged> 0.5,rule,-0.927030,0.289666


In [8]:
rule_other = rule_fit_pcc.fitted_[3].get_rules(all_columns)[['rule','type','coef','importance']]
rule_other.to_csv(os.path.join(OUTPUTPATH, 'Other_rules.csv'), index=False)
rule_other

,rule,type,coef,importance
3118,clogp_corona<= 0.73 & conc> 7.5 & dp_core> 61.5 & salt<= 0.2 & corona_mw_total<= 14338.48 & mon_corona_mv> 0.7838,rule,-1.545762,0.676379
2071,core_mw_total<= 30217.0 & dp_core> 127.0 & mon_corona_psa<= 65.0 & corona_mw_total> 5957.905 & sphere<= 0.5,rule,4.156497,0.520854
230,core_mv_total> 261.2925 & mon_core_psa<= 46.5679,rule,-0.843226,0.419295
2915,mon_core_psa<= 46.5679 & clogp_corona<= 0.73 & dp_corona<= 120.0 & conc> 11.25 & mon_corona_mv> 0.7838,rule,-0.796986,0.397497
972,ratio_vol<= 0.3356 & clogp_core<= 1.4 & clogp_corona<= 0.73 & dp_corona<= 95.5 & sphere<= 0.5 & corona_mw_total<= 14063.4102 & salt<= 0.15,rule,-0.795232,0.389753
2430,conc<= 18.75 & corona_mw_total<= 5113.05 & ratio_mass<= 0.3479 & mon_core_mw<= 151.085 & corona_mw_total<= 14338.48,rule,-1.311937,0.356972
688,temp> 53.0 & salt<= 0.2 & clogp_core<= 1.1101,rule,-0.705300,0.332701
2194,corona_mw_total<= 20078.7627 & clogp_corona<= 0.73 & mon_core_psa<= 46.5755 & conc> 11.25 & mon_corona_mv> 0.7838,rule,-0.627583,0.313551
1495,mon_corona_mw> 162.1158 & ratio_vol<= 0.2363 & corona_mv_total> 39.607 & mon_corona_psa<= 79.32 & mon_corona_apol> 23.4875 & ratio_vol> 0.1885 & sphere<= 0.5,rule,3.535543,0.311278
2366,core_mw_total<= 33858.7998 & core_mw_total> 18010.0 & clogp_corona> 0.125 & corona_mv_total<= 107.5755 & mon_core_apol<= 24.2743 & conc> 17.5,rule,2.792009,0.286438


__Get all morphology rules__


In [9]:
# get all rules:
sphere_top5_rule = rule_sphere
worm_top5_rule = rule_worm
vesicle_top5_rule = rule_vesicle

sphere_top5_rule.reset_index(drop=True, inplace=True)
worm_top5_rule.reset_index(drop=True, inplace=True)
vesicle_top5_rule.reset_index(drop=True, inplace=True)

top5_rule = pd.concat([sphere_top5_rule, worm_top5_rule, vesicle_top5_rule], axis=1)
upper_columns = ['Sphere', 'Worm', 'Vesicle']
lower_columns = ['rule', 'type', 'coef', 'importance']

top5_rule.columns = pd.MultiIndex.from_product([upper_columns, lower_columns], names=['Phase', 'Property'])
top5_rule.to_csv(os.path.join(OUTPUTPATH, 'Combine_All_Rules.csv'), index=False)
top5_rule

Phase                                                                                                                                                                                                                                                                                                                        Sphere  \
Property                                                                                                                                                                                                                                                                                                                       rule   
0         dp_core> 132.5 & mon_core_mv> 1.0413 & clogp_core<= 1.37 & mon_core_mv<= 1.1928 & dp_corona<= 64.5 & core_mw_total> 21611.625 & ratio_vol<= 0.2233 & mon_corona_apol<= 27.5791                                                                                                                                              
1         corona_mw_total<= 11925.2148 & mon_corona_apol<= 23.6109 & core_mw_total> 15703.46 & clogp_core<= 1.37 & dp_corona<= 63.5 & dp_core> 96.5 & mon_corona_mv<= 1.344 & core_mv_total<= 510.1229 & corona_mv_total<= 67.7817                                                                                                    
2         conc<= 15.5 & ratio_mass<= 0.4196 & mon_core_mw<= 145.3096 & ratio_vol<= 0.6272 & conc<= 16.5 & mon_core_mw<= 144.5526 & corona_mw_total> 10396.3247                                                                                                                                                                        
3         ratio_vol<= 0.2732 & corona_mw_total<= 12760.5298 & clogp_core<= 1.37 & mon_core_psa> 46.1575 & mon_core_psa> 46.35 & core_mv_total> 141.3628 & ratio_vol<= 0.3104 & conc> 16.1 & ratio_mass> 0.104 & clogp_corona<= 0.125 & mon_corona_mw<= 287.115                                                                        
4         ratio_vol<= 0.3004 & mon_core_psa> 30.915 & clogp_core<= 1.37 & mon_corona_apol<= 58.2307 & mon_corona_mv> 1.3573 & ratio_mass> 0.1869 & dp_corona> 28.0 & dp_corona<= 64.5 & corona_mv_total<= 78.3154                                                                                                                     
5         mon_corona_mw> 163.9357 & conc> 15.5 & core_mw_total> 30761.0801 & clogp_corona<= -2.73 & mon_corona_psa<= 215.065 & mon_corona_mv<= 2.3358 & conc> 16.1 & ratio_vol<= 0.2996 & clogp_core> 0.895 & core_mw_total> 26510.7197                                                                                               
6         core_mw_total> 27374.7246 & corona_mw_total<= 11925.2148 & corona_mw_total<= 12760.5298 & temp> 47.0 & clogp_core<= 1.37 & conc> 6.25 & ratio_vol<= 0.3038 & ratio_mass<= 0.2783 & mon_corona_apol<= 58.2307 & mon_core_mw<= 145.3096 & clogp_corona> -5.23 & salt<= 0.05 & charged<= 0.5                                   
7         conc<= 27.5 & mon_corona_psa> 58.045 & corona_mw_total<= 12760.5298 & corona_mw_total<= 10396.3247 & mon_core_mw<= 145.3096 & mon_corona_mv<= 1.3573 & charged<= 0.5 & ratio_mass<= 0.3269                                                                                                                                  
8         conc> 14.25 & ratio_vol<= 0.3025 & dp_corona> 28.5 & core_mw_total> 15703.46 & corona_mw_total<= 10497.1396 & mon_core_psa> 40.26 & clogp_corona<= 0.605 & core_mw_total<= 39320.0 & core_mv_total> 152.3496 & dp_corona<= 62.5                                                                                             
9         core_mv_total<= 296.8425 & clogp_core<= 1.18 & conc<= 27.5 & dp_corona<= 87.5 & dp_core<= 185.0 & mon_core_mw<= 145.3096 & ratio_vol<= 0.3115 & mon_corona_mv<= 1.3573 & ratio_vol> 0.2368 & mon_core_mw<= 144.4899                                                                                                         
10        clogp_core<= 1.37 & conc> 6.25 & ph> 2.625 & corona_mw_total<= 156

__Get the Top5 morphology rules__


In [10]:
# get top 5:
sphere_top5_rule = rule_sphere.iloc[:5, ]
worm_top5_rule = rule_worm.iloc[:5, ]
vesicle_top5_rule = rule_vesicle.iloc[:5, ]

sphere_top5_rule.reset_index(drop=True, inplace=True)
worm_top5_rule.reset_index(drop=True, inplace=True)
vesicle_top5_rule.reset_index(drop=True, inplace=True)

top5_rule = pd.concat([sphere_top5_rule, worm_top5_rule, vesicle_top5_rule], axis=1)
upper_columns = ['Sphere', 'Worm', 'Vesicle']
lower_columns = ['rule', 'type', 'coef', 'importance']

top5_rule.columns = pd.MultiIndex.from_product([upper_columns, lower_columns], names=['Phase', 'Property'])
top5_rule.to_csv(os.path.join(OUTPUTPATH, 'Combine_Top5_Rules.csv'), index=False)
top5_rule

Phase                                                                                                                                                                                                                                                   Sphere  \
Property                                                                                                                                                                                                                                                  rule   
0         dp_core> 132.5 & mon_core_mv> 1.0413 & clogp_core<= 1.37 & mon_core_mv<= 1.1928 & dp_corona<= 64.5 & core_mw_total> 21611.625 & ratio_vol<= 0.2233 & mon_corona_apol<= 27.5791                                                                         
1         corona_mw_total<= 11925.2148 & mon_corona_apol<= 23.6109 & core_mw_total> 15703.46 & clogp_core<= 1.37 & dp_corona<= 63.5 & dp_core> 96.5 & mon_corona_mv<= 1.344 & core_mv_total<= 510.1229 & corona_mv_total<= 67.7817                               
2         conc<= 15.5 & ratio_mass<= 0.4196 & mon_core_mw<= 145.3096 & ratio_vol<= 0.6272 & conc<= 16.5 & mon_core_mw<= 144.5526 & corona_mw_total> 10396.3247                                                                                                   
3         ratio_vol<= 0.2732 & corona_mw_total<= 12760.5298 & clogp_core<= 1.37 & mon_core_psa> 46.1575 & mon_core_psa> 46.35 & core_mv_total> 141.3628 & ratio_vol<= 0.3104 & conc> 16.1 & ratio_mass> 0.104 & clogp_corona<= 0.125 & mon_corona_mw<= 287.115   
4         ratio_vol<= 0.3004 & mon_core_psa> 30.915 & clogp_core<= 1.37 & mon_corona_apol<= 58.2307 & mon_corona_mv> 1.3573 & ratio_mass> 0.1869 & dp_corona> 28.0 & dp_corona<= 64.5 & corona_mv_total<= 78.3154                                                

Phase                                \
Property  type      coef importance   
0         rule -2.320588  0.750651    
1         rule -1.996514  0.632467    
2         rule  1.671920  0.533947    
3         rule -1.699665  0.527928    
4         rule -2.869156  0.521525    

Phase                                                                                                                                                                                                                                                                                           Worm  \
Property                                                                                                                                                                                                                                                                                        rule   
0         dp_corona> 27.0 & ratio_vol> 0.2022 & temp> 47.0 & ph> 5.5 & ratio_vol> 0.2135 & corona_mv_total> 79.0114 & conc> 11.5 & dp_core> 157.5 & ratio_mass<= 0.4183 & dp_corona<= 88.0 & mon_core_mw<= 156.06                                                                                      
1         mon_core_psa> 37.18 & mon_corona_apol> 11.8462 & clogp_core<= 1.4 & clogp_core<= 1.15 & ratio_mass> 0.1534 & dp_core<= 146.5 & ratio_mass<= 0.4161 & corona_mv_total<= 52.4219 & core_mw_total<= 43151.2109 & corona_mw_total<= 7143.2949 & ratio_mass> 0.1356 & corona_mv_total<= 69.0254   
2         clogp_core<= 1.18 & mon_core_psa<= 46.7805 & dp_core> 200.5 & conc<= 11.25 & dp_core> 167.5                                                                                                                                                                                                  
3         dp_core> 299.5 & ratio_vol<= 0.19 & dp_corona> 25.5 & ratio_mass<= 0.2363                                                                                                                                                                                                                    
4         core_mv_total> 133.3125 & ratio_vol> 0.2088 & mon_core_apol> 21.0474 & clogp_core<= 1.37 & corona_mw_total<= 16666.9927 & ratio_mass>